In [107]:
from coffea.nanoevents import BaseSchema, NanoEventsFactory
import dask_awkward as dak
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
import hist
import mplhep as hep
import numba
import vector
vector.register_awkward()

In [119]:
events = NanoEventsFactory.from_root(
    file="../data/p8_ee_ZH_ecm240/events_101027117.root:events",
    schemaclass=BaseSchema,
    # schemaclass=DelphesSchema, Doesn't work 
    # schemaclass=NanoAODSchema.v7, Doesn't work
    metadata={
        "path":"/eos/experiment/fcc/ee/generation/DelphesEvents/spring2021/IDEA/p8_ee_ZH_ecm240/events_101027117.root",
        "name":"p8_ee_ZH_ecm240",
        "process":"ZH to mumu, ecm=240 GeV"
    }
).events()

In [120]:
from coffea.analysis_tools import PackedSelection

In [121]:
def index_mask(input_array, index_array):
    '''
    This function matches the given attribute of ReconstructedParticles (for example energy) to the particle index (for example Muon or Electron)
    '''
    # input_array = input_array.compute()
    # index_array = index_array.compute()
    if len(input_array) != len(index_array) :
        raise Exception(f'Length of input_array({len(input_array)}) and index_array({len(index_array)}) does not match!')
    counts = len(index_array)
    
    @numba.jit
    def numba_wrap(input_array, index_array, counts):
        output_array = []
        for event_index in range(counts):
            event_mask = index_array[event_index]
            reco_list = input_array[event_index]
            output_array.append([reco_list[i] for i in  event_mask])
        return output_array
    out = ak.Array(numba_wrap(input_array,index_array,counts))
    
    return out

In [123]:
cut = PackedSelection()

# Filter out any event with no reconstructed particles
Recon = events['ReconstructedParticles/ReconstructedParticles.energy'].compute()
useful_events = events[ak.num(Recon) > 0]


# Generate Reconstructed Particle Attributes
Reco_E = useful_events['ReconstructedParticles/ReconstructedParticles.energy'].compute()
Reco_px = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.x'].compute()
Reco_py = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.y'].compute()
Reco_pz = useful_events['ReconstructedParticles/ReconstructedParticles.momentum.z'].compute()
Reco_q = useful_events['ReconstructedParticles/ReconstructedParticles.charge'].compute()
Reco_mass = useful_events['ReconstructedParticles/ReconstructedParticles.mass'].compute()
cut.add('At least one Reconstructed Particle', ak.all(Reco_E > 0, axis=1))#.compute())

# Generate Muon Attributes
Muon_index = useful_events['Muon#0/Muon#0.index'].compute()
Muon_E = index_mask(Reco_E,Muon_index)
Muon_px = index_mask(Reco_px,Muon_index)
Muon_py = index_mask(Reco_py,Muon_index)
Muon_pz = index_mask(Reco_pz,Muon_index)
Muon_q = index_mask(Reco_q,Muon_index)
Muon_mass = index_mask(Reco_mass,Muon_index)

# Create Array of Muon Lorentz Vector 
Muon = ak.zip({"px":Muon_px,"py":Muon_py,"pz":Muon_pz,"E":Muon_E,"q":Muon_q,}, with_name="Momentum4D")

# Muon pt > 10
Muon_pt_cut = ak.all(Muon.pt > 10, axis=1)
Muon = Muon[Muon_pt_cut]
cut.add('Muon $p_T$ > 10 [GeV]',Muon_pt_cut)

In [124]:
cut.names

['At least one Reconstructed Particle', 'Muon $p_T$ > 10 [GeV]']

In [125]:
cut.cutflow(*cut.names).print()

Cutflow stats:
Cut At least one Reconstructed Particle:pass = 99984               cumulative pass = 99984               all = 99984               -- eff = 100.0 %                    -- cumulative eff = 100.0 %
Cut Muon $p_T$ > 10 [GeV]:pass = 97219               cumulative pass = 97219               all = 99984               -- eff = 97.2 %                    -- cumulative eff = 97.2 %
